# CDO API exploration

[CDO endpoints](https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted)

[GHCN (Global Historical Climatology Network) – Daily Documentation](https://www.ncei.noaa.gov/data/daily-summaries/doc/GHCND_documentation.pdf)

[Go Code in Jupyter Notebook](https://github.com/janpfeifer/gonb/blob/main/examples/tutorial.ipynb)

1. get_request is a function that takes a URL as an argument, sends an HTTP GET request to that URL, and returns the body of the response as a string. The "token" in the request header is CDO API key

2. The cache.Cache function is being used to create cached versions of the API responses. If the cache for a given key (the first argument) does not exist or is outdated, the function passed as the second argument will be called to update the cache. The function being passed is the get_request function, with the URL for the API request.

3. The cache.Cache function returns the cached response, or updates the cache and returns the new response if the cache is outdated.

4. Multiple cached API responses are being stored in global variables (daily_summaries, location_categories, cities_locations, zip_code_locations, etc). Each variable corresponds to a different API endpoint.

Please replace "Your-Token" with the actual token, which you should have received from the CDO API provider. 

One more thing to note is gonb provides a short cut: "%%" to avoid having to write func main() at every cell

In [ ]:
import (
	"encoding/json"
	"fmt"
	"io/ioutil"
	"net/http"
	"github.com/janpfeifer/gonb/cache"
)

func get_request(url string) string{
	req, err := http.NewRequest("GET", url, nil)
	if err != nil {
		fmt.Println(err)
		return ""
	}

	// Replace "Your-Token" with your actual token.
	req.Header.Set("token", "Your-Token")

	resp, err := http.DefaultClient.Do(req)
	if err != nil {
		fmt.Println(err)
		return ""
	}
	defer resp.Body.Close()

	body, err := ioutil.ReadAll(resp.Body)
	if err != nil {
		fmt.Println(err)
		return ""
	}
	return string(body)
}

var (
    daily_summaries = cache.Cache("GHCND", func() string { return get_request("https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets/GHCND") })
	data_categories = cache.Cache("data-categories", func() string { return get_request("https://www.ncei.noaa.gov/cdo-web/api/v2/datacategories?datasetid=GHCND") })
	data_types = cache.Cache("data-types", func() string { return get_request("https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GHCND&datacategoryid=TEMP") })
	location_categories = cache.Cache("location_categories", func() string { return get_request("https://www.ncei.noaa.gov/cdo-web/api/v2/locationcategories") })
	cities_locations = cache.Cache("locations-cit1y", func() string { return get_request("https://www.ncei.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY&sortfield=name&sortorder=desc&limit=100") })
	zip_code_locations = cache.Cache("zip-code-locations", func() string { return get_request("https://www.ncei.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=ZIP&sortfield=name&sortorder=desc&limit=100") })
	data_city = cache.Cache("data-daily-city-sample", func() string { return get_request("https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=CITY:SZ000007&startdate=2022-05-01&enddate=2022-05-01&units=metric&datatypeid=TMAX,TMIN,TAVG") })
	data_station = cache.Cache("data-daily-sample", func() string { return get_request("https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:SZ000003700&startdate=2022-05-01&enddate=2022-05-01&datatypeid=TMAX,TMIN,TAVG&units=metric") })
	stations = cache.Cache("stations-GHCND", func() string { return get_request("https://www.ncei.noaa.gov/cdo-web/api/v2/stations?datasetid=GHCND&limit=100") })
)

%%
fmt.Println(stations)